In [121]:
import pandas as pd
import numpy as np

#READ data frame 
df_input = pd.read_csv("adult.input")
df_test = pd.read_csv("adult.test")

#add the column names
df_input.columns = ['age','type_employer','fnlwgt','education','education_num','marital','occupation',
             'relationship','race','sex','capital_gain','capital_loss','hr_per_week','country',
             'income']
df_test.columns = ['age','type_employer','fnlwgt','education','education_num','marital','occupation',
             'relationship','race','sex','capital_gain','capital_loss','hr_per_week','country',
             'income']

#add missing first row, due to column name change
first_row_input = {'age' : '24','type_employer' : 'Private','fnlwgt' : '176189','education' : 'HS-grad',
       'education_num' : '9','marital' : 'Married-civ-spouse','occupation' : 'Machine-op-inspct',
             'relationship' : 'Husband','race' : 'White','sex' : 'Male','capital_gain' : '0',
       'capital_loss' : '0','hr_per_week' : '40','country' : 'United-States',
             'income' : '<=50K'}

df_input = df_input.append(first_row_input,ignore_index = True)

C:\Users\driqu001\AppData\Local\Temp\ipykernel_1564\166307858.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_input = df_input.append(first_row_input,ignore_index = True)


In [122]:
first_row_test = {'age' : '43','type_employer' : 'Self-emp-not-inc','fnlwgt' : '292175','education' : 'Masters',
       'education_num' : '14','marital' : 'Divorced','occupation' : 'Exec-managerial',
             'relationship' : 'Unmarried','race' : 'White','sex' : 'Female','capital_gain' : '0',
       'capital_loss' : '0','hr_per_week' : '45','country' : 'United-States',
             'income' : '?'}
df_test = df_test.append(first_row_test,ignore_index = True)

C:\Users\driqu001\AppData\Local\Temp\ipykernel_1564\4204825365.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(first_row_test,ignore_index = True)


In [123]:

#remove instances with the missing values "?" marker
df_input = df_input.loc[df_input["age"] != "?"]
df_input = df_input.loc[df_input["type_employer"] != "?"]
df_input = df_input.loc[df_input["fnlwgt"] != "?"]
df_input = df_input.loc[df_input["education"] != "?"]
df_input = df_input.loc[df_input["education_num"] != "?"]
df_input = df_input.loc[df_input["marital"] != "?"]
df_input = df_input.loc[df_input["occupation"] != "?"]
df_input = df_input.loc[df_input["relationship"] != "?"]
df_input = df_input.loc[df_input["race"] != "?"]
df_input = df_input.loc[df_input["sex"] != "?"]
df_input = df_input.loc[df_input["capital_gain"] != "?"]
df_input = df_input.loc[df_input["capital_loss"] != "?"]
df_input = df_input.loc[df_input["hr_per_week"] != "?"]
df_input = df_input.loc[df_input["country"] != "?"]
df_input = df_input.loc[df_input["income"] != "?"]

#df_input.shape

In [124]:
#remove attributes fnlwgt,education,relationship
df_input.drop(columns=["fnlwgt","education","relationship"],inplace=True)

#df_input.shape

In [125]:
#binarization of capital gain, capital loss, and native country attributes
df_input.capital_gain[df_input.capital_gain.astype(int) > 0] = 1
df_input.capital_gain[df_input.capital_gain.astype(int) == 0] = 0

df_input.capital_loss[df_input.capital_loss.astype(int) > 0] = 1
df_input.capital_loss[df_input.capital_loss.astype(int) == 0] = 0


In [126]:
df_input["country"] = np.where(df_input["country"] == "United-States", 1, 0)

#df_input

In [127]:
#Discretization of continuous attributes [age,hours_per_week]
df_input["young"] = np.where(df_input.age.astype(int) <= 25, 1, 0)
df_input["adult"] = np.where((df_input.age.astype(int) >= 26) & (df_input.age.astype(int) <= 45), 1, 0)
df_input["senior"] = np.where((df_input.age.astype(int) >= 46) & (df_input.age.astype(int) <= 65), 1, 0)
df_input["old"] = np.where((df_input.age.astype(int) >= 66) & (df_input.age.astype(int) <= 90), 1, 0)

df_input.drop(columns=["age"],inplace=True)
#df_input

In [128]:
df_input["part_time"] = np.where(df_input.hr_per_week.astype(int) < 40, 1, 0)
df_input["full_time"] = np.where(df_input.hr_per_week.astype(int) == 40, 1, 0)
df_input["over_time"] = np.where(df_input.hr_per_week.astype(int) > 40, 1, 0)
df_input.drop(columns=["hr_per_week"],inplace=True)

# df_input

In [129]:
# Merge values and creation of new binary assymetric attributes, WORKING CLASS
# [Fed,local,stat]-> gov
# [w/o pay, never worked] -> not_working
# [Private] -> private
# [Self-inc,self-not-inc] -> self_employed

df_input["gov"] = np.where( ((df_input["type_employer"] == "Federal-gov") | (df_input["type_employer"] == "Local-gov") | (df_input["type_employer"] == "State-gov")), 1, 0)
df_input["not_working"] = np.where( ((df_input["type_employer"] == "Without-pay") | (df_input["type_employer"] == "Never-worked")), 1, 0)
df_input["private"] = np.where( (df_input["type_employer"] == "Private"), 1, 0)
df_input["self_employed"] = np.where( ((df_input["type_employer"] == "Self-emp-inc") | (df_input["type_employer"] == "Self-emp-not-inc")), 1, 0)

df_input.drop(columns=["type_employer"],inplace=True)

# df_input

In [130]:
# Merge values and creation of new binary assymetric attributes, MARITAL STATUS
# [Married-AF-spouse, Married-civ-spouse)]-> married
# [w/o pay, never worked] -> not_working
# [Never-married] -> never_married
# [Married-spouse-absent, Separated, Divorced, Widowed] -> not-married

df_input["married"] = np.where( ((df_input["marital"] == "Married-AF-spouse") | (df_input["marital"] == "Married-civ-spouse") ), 1, 0)
df_input["never_married"] = np.where( (df_input["marital"] == "Never-married" ), 1, 0)
df_input["not_married"] = np.where( ((df_input["marital"] == "Married-spouse-absent") | (df_input["marital"] == "Separated") 
| (df_input["marital"] == "Divorced") | (df_input["marital"] == "Widowed")), 1, 0)

df_input.drop(columns=["marital"],inplace=True)

# df_input

In [131]:
# Merge values and creation of new binary assymetric attributes, OCCUPATION
# [Exec-managerial]-> exec_managerial
# [Prof-specialty] -> prof_specialty
# [Tech-support, Adm-clerical, Priv-house-serv, Protective-serv, Armed-Forces, Other-service] -> other
# [Craft-repair, Farming-fishing, Handlers-cleaners, Machine-op-inspct, Transport-moving] -> manual_work
# [Sales] -> manual_work
